### Example for retrieving feature store records from SageMaker Endpoint
Assumptions - Must have one Feature Group Provisioned

Custom inference file

In [15]:
%%writefile inference.py

import json
import os
import pickle as pkl
import time
import sys
import subprocess
import numpy as np

subprocess.check_call([sys.executable, "-m", "pip", "install", "sagemaker"])

import boto3
import sagemaker

import sagemaker_xgboost_container.encoder as xgb_encoders

boto_session = boto3.Session()
boto_fs_client = boto_session.client(service_name='sagemaker-featurestore-runtime')

def model_fn(model_dir):
    print ('processing - in model_fn')
    return None


def input_fn(request_body, request_content_type):
    print ('processing - in input_fn')
    return request_body


def predict_fn(input_data, model):
    print ('processing - in predict_fn')
    
    params = input_data.split(',')
    fg_name = params[0]
    input_feat_id = int(params[1])
    
    
    start = time.time()
    rec = boto_fs_client.get_record(FeatureGroupName=fg_name, RecordIdentifierValueAsString=str(input_feat_id))
    end = time.time()
    feat = rec.get('Record', None)
    if not feat:
        print (f'processing - unable to read feature record with id {input_feat_id}, for fg {fg_name}')
    else:
        resp_feat_id = feat[0]['ValueAsString']
        print (f'processing - successfull get of feature record with id {resp_feat_id}, in fg {fg_name}')
    duration = end-start
    
    print (f'processing - duration = {duration}')
    
    return duration


def output_fn(prediction, content_type):
    print ('processing - output_fn', prediction)
    return str(prediction)


Overwriting inference.py


Deploy/Create a sagemaker endpoint using xgboost model

In [16]:
import sagemaker
from sagemaker.xgboost.model import XGBoostModel
from sagemaker.serializers import CSVSerializer

role = sagemaker.get_execution_role()



xgboost_inference_model = XGBoostModel(
    model_data="s3://fs-get-record/sagemaker/xgboost_credit_risk/output/xgboost-2021-06-02-14-55-49-198/output/model.tar.gz",
    role=role,
    entry_point="inference.py",
    framework_version="1.2-2"
)

predictor = xgboost_inference_model.deploy(
    instance_type='ml.c4.xlarge',
    initial_instance_count=1,
    serializer=CSVSerializer(),
)

---------------!

Single run, the predictor returns a duration (secs). The feature group should be provisioned with the id

In [ ]:
fg_name = 'ingest-fg-05-25-2021-17-24-43'
feat_id = 20000
req = f'{fg_name}, {feat_id}'
resp = predictor.predict(req)
duration = resp[0][0]
duration

Helper function to print stats

In [17]:
def get_stats(times):
    import numpy as np
    a = np.array(times)
    p_50 = np.percentile(a, 50)
    p_95 = np.percentile(a, 95)
    p_99 = np.percentile(a, 99)
    avg = np.average(a)
    
    return {
        'avg':avg, 
        'p_50': p_50, 
        'p_95': p_95, 
        'p_99': p_99}

In [18]:
fg_name = 'ingest-fg-05-25-2021-17-24-43'
feat_id = 20000
times = []
for i in range(10000):
    req = f'{fg_name}, {feat_id}'
    resp = predictor.predict(req)
    duration = float(resp[0][0])
    times.append(duration)
    
get_stats(times)

{'avg': 0.009953949403762817,
 'p_50': 0.008751869201660156,
 'p_95': 0.011943900585174548,
 'p_99': 0.03251652240753191}

In [19]:
get_stats(times[1:])

{'avg': 0.009944982177699277,
 'p_50': 0.008751869201660156,
 'p_95': 0.011942887306213379,
 'p_99': 0.03250744342803955}

Create Feature Group, provision sample data and retrieve, to simulate cold starts

In [ ]:
import sagemaker
import pandas as pd
sm_session = sagemaker.Session()
role = sagemaker.get_execution_role()

Generate Sample Data

In [ ]:
num_fgs = 25
data_feat_ids = [i for i in range(1, num_fgs+1)]
data_feat_eventtimes = ['2020-12-21T01:00:00Z' for i in range(1, num_fgs+1)]
data_feat_qtys = [i *10 for i in range(1, num_fgs+1)]
data_fg_names = ['test-fg-'+str(i) for i in range(1, num_fgs+1)]

Helpers to provision and ingest sample data into feature groups

In [ ]:
from sagemaker.feature_store.feature_group import FeatureGroup
import time
def cast_object_to_string(df):
    for col in df.columns:
        if df.dtypes[col] == 'object':
            df[col] = df[col].astype('str').astype('string')
            
def wait_for_feature_group_creation_complete(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
#        print("Waiting for Feature Group Creation")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    if status != "Created":
        raise RuntimeError(f"Failed to create feature group {feature_group.name}")
#   print(f"FeatureGroup {feature_group.name} successfully created.")

def ingest_data(seed_data):
    for feat_id, feat_time, feat_qty, fg_name in seed_data:
        tmp = {
            'feat_id': [feat_id],
            'feat_time': [feat_time],
            'feat_Qty': [feat_qty]
        }
        df = pd.DataFrame(data=tmp)
        # cast object dtype to string. The SageMaker FeatureStore Python SDK will then map the string dtype to String feature type.
        cast_object_to_string(df)
        fg = FeatureGroup(name=fg_name, sagemaker_session=sm_session)
        fg.load_feature_definitions(data_frame=df)
        

        kwargs = dict(
            s3_uri = False,
            record_identifier_name='feat_id',
            event_time_feature_name='feat_time',
            role_arn=role,
            enable_online_store=True
        )
        fg.create(**kwargs)
        wait_for_feature_group_creation_complete(fg)
        
        fg.ingest(data_frame=df, wait=True)

In [ ]:
sample_data = zip(data_feat_ids, data_feat_eventtimes, data_feat_qtys, data_fg_names)
ingest_data(sample_data)    

Call get record for newly created feature groups to simulate cold times

In [ ]:
times = []
for i in range(len(data_fg_names)):
    fg_name = data_fg_names[i]
    feat_id = data_feat_ids[i]
    req = f'{fg_name}, {feat_id}'
    resp = predictor.predict(req)
    duration = float(resp[0][0])
    times.append(duration)
    
get_stats(times)

In [ ]:
#delete feature groups
for fg_name in data_fg_names:
    fg = FeatureGroup(name=fg_name, sagemaker_session=sm_session)
    fg.delete()

In [ ]:
# delete endpoint
predictor.delete_endpoint()